In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs4
import io
import re

In [2]:
#EMODnet has an API for station list. Below is for getting station list from xml document that saved after running the API. 
#In order for it to be up-to-date list, it is suggested to run the API from time to time and use the new one.

def get_platforms_emodnet():
    
    import xml.etree.ElementTree as et 
    #data = './site_data/WSEmodnet.xml'
    #print(et.fromstring(data.encode('utf-16-le')))
    #data=open("WSEmodnet.xml", "r")
    with open('WSEmodnet.xml', 'r') as file:
        data = file.read()

    #print(data)
    #data = open("WSEmodnet.xml").read()
    #root = et.fromstring(data)
    #data='emodnet.xml'
    #parsed_xml = et.parse(data)
    xml_root = et.fromstring(data)
    #print(xml_root)
    #xml_root = parsed_xml.getroot()

    station_list=pd.DataFrame({'PlatformID':[], 'PlatformName':[], 'Latitude':[], 
                         'Longitude':[], 'Parameters':[], 'YearDataMeasured':[], 
                         'LastDataMeasured':[], 'Institution':[]})

    for platform in xml_root.findall('Platform'):
        pid = platform.find('PlatformID').text
        pcode = platform.find('PlatformCode').text
        latitude = platform.find('Latitude').text
        longitude = platform.find('Longitude').text
        parameters = platform.find('Parameters').text
        parameters = list(parameters.split(";"))
        years = platform.find('YearDataMeasured')
        if not years is None:
            years=years.text
            years = list(years.split(";"))
        lastData = platform.find('LastDataMeasured').text
        institute = platform.find('Institution').text
        station_list = station_list.append({'PlatformID':pid, 'PlatformName':pcode, 'Latitude':latitude, 'Longitude':longitude, 
                                'Parameters':parameters, 'YearDataMeasured':years, 
                                'LastDataMeasured':lastData, 'Institution':institute},ignore_index=True)
    #station_list.drop_duplicates(subset='PlatformID', inplace=True)    
    #station_list.set_index('PlatformID', drop=False, inplace=True)
    station_list.reset_index(inplace=True, drop=True)
    return station_list


In [3]:
#ParameterID:181;	ParameterGroup:Waves;	Code:WV_GIG_PER;	Descr:dominant/peak period  ;	UnitaMisura:s;	
319        
#ParameterID:152;	ParameterGroup:Waves;	Code:SWHT;	CFStandardName:;	Descr:swell height;	UnitaMisura:meter;	
192
#ParameterID:153;	ParameterGroup:Waves;	Code:SWPR;	CFStandardName:;	Descr:swell period;	UnitaMisura:second;	
80
#ParameterID:221;	ParameterGroup:Waves;	Code:VZMX;	CFStandardName:sea_surface_wave_maximum_height;	Descr:Maximum zero crossing wave height (Hmax);	UnitaMisura:meter;	
146

146

In [7]:
#Comparison within emodnet stations list

emodnet_list=get_platforms_emodnet()

varlist=['VHM0','VTDH','VGHS']
varlist2=['VTPK','VTZA','VGTA']

variables=['VDIR','VHM0','VPED','VTDH','VTPK','VTZA','VGHS','VGTA','VHZA','VMDR','VTM02','VTM10']

variables=['VHM0','VTDH'] #63 common
variables=['VHM0','VGHS'] #5 common
variables=['VGHS','VTDH'] #9 common

#variables=['VTM02','VGTA'] #2 common
#variables=['VTPK','VGTA'] #9 common
variables=['VTZA','VGTA']  #10 common
variables=['VGTA']
variables=['VGHS']

#print(station_list['Parameter'].apply(lambda x: x ** 2))
#print(station_list['Parameters'].loc[(station_list['Parameters'].apply(lambda x: x.isin(variables)).any())])
listaa=list()

for i,r in emodnet_list.iterrows():
    if all(param in r['Parameters'] for param in variables):
        vars2=['VHZA']
        if any(other in r['Parameters'] for other in vars2):
            listaa.append(r)
df_vars = pd.DataFrame(listaa)  
print(len(listaa))
#print(df_vars['Latitude'].str.replace(',', '.').astype(float).describe())
print(df_vars.index)

0
Index([], dtype='object')


In [5]:
#Comparison within emodnet stations list

variables=['VHM0','VTDH','VTPK','VTZA','VGHS','VGTA','VHZA','VTM02','VTM10','VCMX','VTMX','VZMX','VTZM','SWHT','VEPK']

#Code:VDIR;	CFStandardName:sea_surface_wave_from_direction; #Descr:wave direction rel. true north;
#241
#Code:VHM0;	CFStandardName:sea_surface_wave_significant_height; #Descr:spectral signif. wave height;
#638
#Code:VPED;	CFStandardName:sea_surface_wave_from_direction_at_variance_spectral_density_maximum; #Descr:wave spectrum peak energy dir.;
#141
#Code:VTDH;	CFStandardName:sea_surface_wave_significant_height; #Descr:significant wave height;
#468
#Code:VTPK;	CFStandardName:sea_surface_wave_period_at_variance_spectral_density_maximum; #Descr:wave spectrum peak period;
#626
#Code:VTZA;	CFStandardName:sea_surface_wave_zero_upcrossing_period; #Descr:aver zero crossing wave period;
#250
#Code:VGHS;	CFStandardName:sea_surface_wave_significant_height; #Descr:Generic significant wave height (Hs);
#235
#Code:VGTA;	CFStandardName:sea_surface_wave_mean_period; #Descr:Generic average wave period;
#72
#Code:VHZA;	CFStandardName:sea_surface_wave_mean_height; #Descr:Average zero crossing wave height (Hzm)
#24
#Code:VMDR;	CFStandardName:sea_surface_wave_from_direction; #Descr:Mean wave direction from (Mdir);
#90
#Code:VTM02;	CFStandardName:sea_surface_wave_mean_period_from_variance_spectral_density_second_frequency_moment; #Descr:Spectral moments (0,2) wave period (Tm02);
#147
#Code:VTM10;	CFStandardName:sea_surface_wave_mean_period_from_variance_spectral_density_inverse_frequency_moment; #Descr:Spectral moments (-1,0) wave period (Tm-10);
#27

variables=['VDIR','VHM0','VPED','VTDH','VTPK','VTZA','VGHS','VGTA','VHZA','VMDR','VTM02','VTM10']

variables=['VHM0','VTDH'] #63 common

variables=['VTDH','VGHS'] #9 common
variables=['VHM0','VGHS'] #5 common
variables=['VAVH','VGHS'] #8 common

variables=['VHM0','VHZA'] #16 common
variables=['VTDH','VHZA'] #2 common
variables=['VGHS','VHZA'] #0 common

variables=['VDIR','VPED'] #54 common
variables=['VDIR','VMDR'] #52 common

#most existing commonly: VHM0 and VTPK
varlist=['VHM0']
varlist2=['VTPK','VTZA','VGTA']
#print(station_list['Parameter'].apply(lambda x: x ** 2))
#print(station_list['Parameters'].loc[(station_list['Parameters'].apply(lambda x: x.isin(variables)).any())])
listaa=list()

for i,r in emodnet_list.iterrows():
    if any(param in r['Parameters'] for param in varlist):
        if any(param in r['Parameters'] for param in varlist2):
            listaa.append(r)
df_vars = pd.DataFrame(listaa)  
print(len(listaa))
print(df_vars)

#for i,r in station_list.iterrows():
#    if all(param in r['Parameters'] for param in variables):
#        listaa.append(r)
#df_vars = pd.DataFrame(listaa)  
#print(len(listaa))
#print(df_vars)



629
     PlatformID        PlatformName  Latitude Longitude  \
1           434        HuvudskarOst    58,936   19,1598   
2           479               ATHOS    39,975   24,7294   
3           493                Bath      51,4      4,21   
4           502           Hansweert   51,4466    3,9977   
5           531  LichteilandGoeree1  51,92583   3,66972   
...         ...                 ...       ...       ...   
1567     988855           Trapegeer    51,138     2,583   
1568     988860            ZwinBuoy    51,372     3,359   
1569     989004                 N7S  54,26205   6,35623   
1583     995607          Raversijde     51,22     2,869   
1587     998724             6201081  37,69133  -9,71622   

                                             Parameters  \
1     [PSAL, TUR4, VEMH, DEPH, TUR2, VTPK, DOX1, VMD...   
2     [DEPH, ATMS, DOX1, DRYT, FLU2, GSPD, HCDT, HCS...   
3      [DEPH, VAVH, VHM0, VAVT, VTZA, SWHT, VTPK, ASLV]   
4      [DEPH, VAVH, VHM0, VAVT, VTZA, VTPK, SWHT, A

In [6]:
#emodnet eliminate stations does not contain any of the list vars

varlist=['VHM0','VTDH','VGHS','VTPK','VTZA','VGTA']

listaa=list()

for i,r in emodnet_list.iterrows():
    if not any(param in r['Parameters'] for param in varlist):
        continue
    else:
        listaa.append(r)
df_vars2 = pd.DataFrame(listaa)  
print(len(listaa))
print(df_vars2)

#for i,r in station_list.iterrows():
#    if all(param in r['Parameters'] for param in variables):
#        listaa.append(r)
#df_vars = pd.DataFrame(listaa)  
#print(len(listaa))
#print(df_vars)



1301
     PlatformID  PlatformName  Latitude Longitude  \
0           430   Finngrundet      60,9   18,6167   
1           434  HuvudskarOst    58,936   19,1598   
2           479         ATHOS    39,975   24,7294   
3           493          Bath      51,4      4,21   
4           502     Hansweert   51,4466    3,9977   
...         ...           ...       ...       ...   
1585     997395       2101758    43,784   139,318   
1586     997396       2101759    30,237   129,507   
1587     998724       6201081  37,69133  -9,71622   
1588     999604       6402566   65,2562  -32,8282   
1589    1001241       6402567   65,0134   -8,3406   

                                             Parameters  \
0      [TEMP, DEPH, VTDH, VTPK, VTZA, VDIR, WVST, GWDR]   
1     [PSAL, TUR4, VEMH, DEPH, TUR2, VTPK, DOX1, VMD...   
2     [DEPH, ATMS, DOX1, DRYT, FLU2, GSPD, HCDT, HCS...   
3      [DEPH, VAVH, VHM0, VAVT, VTZA, SWHT, VTPK, ASLV]   
4      [DEPH, VAVH, VHM0, VAVT, VTZA, VTPK, SWHT, ASLV]   
... 

In [7]:
#ADJUSTED variables does not exist in the station list.. 

dbID = 'EP_ERD_INT_%s_AL_TS_NRT.subset?EP_PLATFORM_ID,EP_PLATFORM_CODE,EP_PLATFORM_TYPE&.viewDistinctData=10000&.viewRelatedData=0&distinct()'


In [8]:
def get_platforms_erddap(param):
    url='https://erddap.emodnet-physics.eu/erddap/tabledap/'+dbID % param
    response = requests.get(url, verify=False).text

    platforms = pd.read_html(str(response),skiprows=[1],header=[0],attrs={"class":"erd commonBGColor nowrap"})[0]
    platforms.drop_duplicates(subset='EP_PLATFORM_ID', inplace=True)
    #platforms.set_index('EP_PLATFORM_ID',inplace=True)
    platforms.columns=['PlatformID','PlatformName','PlatformType']
    #platforms['PlatformID']=platforms['PlatformID'].astype(str)
    #platforms.set_index('PlatformID', drop=False, inplace=True)
    platforms.reset_index(inplace=True, drop=True)
    return platforms

In [9]:
#Erddap db platforms

vhm0=get_platforms_erddap('VHM0')
vtdh=get_platforms_erddap('VTDH')
vghs = get_platforms_erddap('VGHS')
vhza = get_platforms_erddap('VHZA')

tz = get_platforms_erddap('VTZA')
tp = get_platforms_erddap('VTPK')
vgta = get_platforms_erddap('VGTA')
vtm02 = get_platforms_erddap('VTM02')

varlist=['VHM0','VTDH','VGHS']
varlist2=['VTPK','VTZA','VGTA']

print((vhm0))
print('vtdh '+str(len(vtdh)))
print('vghs '+str(len(vghs)))
print('vtza-tz '+str(len(tz)))
print('vtpk-tp '+str(len(tp)))
print('vgta '+str(len(vgta)))
print('vhza '+str(len(vhza)))
print(len(vtm02))

/opt/tljh/user/lib/python3.6/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/opt/tljh/user/lib/python3.6/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/opt/tljh/user/lib/python3.6/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/opt/tljh/user/lib/python3.6/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified

     PlatformID    PlatformName          PlatformType
0       1004029         Kariani                    MO
1         10407       LasoOstWR  MO_VHM0_TS_INSTAC.nc
2         10451         LasoOst  MO_VHM0_TS_INSTAC.nc
3         10586  SodraOstersjon  MO_VHM0_TS_INSTAC.nc
4         10818  HuvudskarOstWR  MO_VHM0_TS_INSTAC.nc
..          ...             ...                   ...
643      988855       Trapegeer                    MO
644      988860        ZwinBuoy                    MO
645      989004             N7S                    MO
646      995607      Raversijde                    MO
647      998724         6201081                    MO

[648 rows x 3 columns]
vtdh 3
vghs 436
vtza-tz 219
vtpk-tp 643
vgta 230
vhza 31
378


In [10]:
#comparison within Erddap db's

common_count=0
diff_count=0

for i,r in vhm0.iterrows():
    if r['PlatformID'] in vtdh.PlatformID.values:
        common_count+=1
    else:
        diff_count+=1
print('For vhm0 & vtdh; common points '+str(common_count)+' different points '+str(diff_count))

common_count=0
diff_count=0

for i,r in vhm0.iterrows():
    if r['PlatformID'] in vghs.PlatformID.values:
        common_count+=1
    else:
        diff_count+=1
print('For vhm0 & vghs; common points '+str(common_count)+' different points '+str(diff_count))

common_count=0
diff_count=0

for i,r in vghs.iterrows():
    if r['PlatformID'] in vtdh.PlatformID.values:
        common_count+=1
    else:
        diff_count+=1
print('For vghs & vtdh; common points '+str(common_count)+' different points '+str(diff_count))

common_count=0
diff_count=0

for i,r in tp.iterrows():
    if r['PlatformID'] in tz.PlatformID.values:
        common_count+=1
    else:
        diff_count+=1
print('For vtpk-tp & vtza-tz ; common points '+str(common_count)+' different points '+str(diff_count))

common_count=0
diff_count=0

for i,r in tz.iterrows():
    if r['PlatformID'] in vgta.PlatformID.values:
        common_count+=1
    else:
        diff_count+=1
print('For vtza-tz & vgta; common points with emodnet station list '+str(common_count)+' different points '+str(diff_count))

common_count=0
diff_count=0

for i,r in vgta.iterrows():
    if r['PlatformID'] in tp.PlatformID.values:
        common_count+=1
    else:
        diff_count+=1
print('For vgta & vtpk-tp; common points with emodnet station list '+str(common_count)+' different points '+str(diff_count))

common_count=0
diff_count=0

for i,r in tp.iterrows():
    if r['PlatformID'] in vtm02.PlatformID.values:
        common_count+=1
    else:
        diff_count+=1
print('For vtpk-tp & vtm02-tz ; common points '+str(common_count)+' different points '+str(diff_count))

common_count=0
diff_count=0

for i,r in vhm0.iterrows():
    if r['PlatformID'] in tp.PlatformID.values:
        common_count+=1
    else:
        diff_count+=1
print('For vtpk-tp & vhm0 ; common points '+str(common_count)+' different points '+str(diff_count))


For vhm0 & vtdh; common points 0 different points 648
For vhm0 & vghs; common points 30 different points 618
For vghs & vtdh; common points 0 different points 436
For vtpk-tp & vtza-tz ; common points 149 different points 494
For vtza-tz & vgta; common points with emodnet station list 12 different points 207
For vgta & vtpk-tp; common points with emodnet station list 47 different points 183
For vtpk-tp & vtm02-tz ; common points 373 different points 270
For vtpk-tp & vhm0 ; common points 594 different points 54


In [11]:
#Comparison of emodnet and erddap stations

varlist=['VHM0','VTDH','VGHS','VTPK','VTZA','VGTA']
allErddapStations = pd.DataFrame()
common= pd.DataFrame()
for var in varlist:
    platforms=get_platforms_erddap(var)
    allErddapStations = allErddapStations.append(platforms, ignore_index=True)
    allErddapStations.drop_duplicates(subset='PlatformID', inplace=True)
    allErddapStations['PlatformID']=allErddapStations['PlatformID'].astype(str)
    allErddapStations.reset_index(inplace=True, drop=True)
print(allErddapStations)

for i,r in df_vars2.iterrows():
    if allErddapStations['PlatformID'].eq(r['PlatformID']).any():
    #if (allErddapStations['PlatformID'] == r['PlatformID']).any():
        #if common.empty:
            #common=r
        #else:
            common = common.append(emodnet_list.loc[i])
print(common)

common= pd.DataFrame()

for i,r in allErddapStations.iterrows():
    if emodnet_list['PlatformID'].eq(r['PlatformID']).any():
    #if (allErddapStations['PlatformID'] == r['PlatformID']).any():
        #if common.empty:
            #common=r
        #else:
        common = common.append(emodnet_list.loc[i])
print(common)

common= pd.DataFrame()

for i,r in emodnet_list.iterrows():
    if allErddapStations['PlatformID'].eq(r['PlatformID']).any():
    #if (allErddapStations['PlatformID'] == r['PlatformID']).any():
        #if common.empty:
            #common=r
        #else:
            common = common.append(emodnet_list.loc[i])
print(common)

/opt/tljh/user/lib/python3.6/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/opt/tljh/user/lib/python3.6/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/opt/tljh/user/lib/python3.6/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/opt/tljh/user/lib/python3.6/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified

     PlatformID    PlatformName          PlatformType
0       1004029         Kariani                    MO
1         10407       LasoOstWR  MO_VHM0_TS_INSTAC.nc
2         10451         LasoOst  MO_VHM0_TS_INSTAC.nc
3         10586  SodraOstersjon  MO_VHM0_TS_INSTAC.nc
4         10818  HuvudskarOstWR  MO_VHM0_TS_INSTAC.nc
...         ...             ...                   ...
1332     965444    CoffsHarbour  MO_VGTA_TS_INSTAC.nc
1333     965446      CrowdyHead  MO_VGTA_TS_INSTAC.nc
1334     965447            Eden  MO_VGTA_TS_INSTAC.nc
1335     965451      PortKembla  MO_VGTA_TS_INSTAC.nc
1336     965453          Sydney  MO_VGTA_TS_INSTAC.nc

[1337 rows x 3 columns]
                                            Institution  LastDataMeasured  \
1     SMHI - Swedish Meteorological and Hydrological...  2020/06/07 16:00   
2     HCMR - Hellenic Centre for Marine Research, In...  2020/06/07 23:00   
3       RWS - Rijkswaterstaat Waterdienst - Netherlands  2020/06/07 23:15   
4       RWS - Rijks

In [12]:
# vhza

df_vhza = (emodnet_list.loc[emodnet_list.Parameters.apply(lambda x: 'VHZA' in x)])
df_vhza.reset_index(inplace=True, drop=True)
df_vhza


,PlatformID,PlatformName,Latitude,Longitude,Parameters,YearDataMeasured,LastDataMeasured,Institution
0,13795,61499,"39,4928","2,7004","[DEPH, ATMP, CNDC, DEWT, DRYT, GSPD, NRAD, PSA...","[2012, 2013, 2014, 2015, 2016, 2017, 2018, 201...",2020/06/07 00:30,SOCIB - Balearic Islands Coastal Observing and...
1,28126,Algeciras-coast-buoy,"36,0657","-5,4149","[DEPH, VHM0, VAVH, VZMX, VHZA, VH110, VTPK, VA...","[2014, 2015, 2016, 2017, 2018, 2019]",2019/10/20 22:00,PdE - Puertos del Estado - Spain
2,28127,Barcelona-coast-buoy,"41,32","2,2","[DEPH, VHM0, VAVH, VZMX, VHZA, VH110, VTPK, VA...","[2004, 2005, 2006, 2007, 2008, 2009, 2010, 201...",2020/06/07 22:00,PdE - Puertos del Estado - Spain
3,28130,Bilbao-coast-buoy,"43,397","-3,1311","[DEPH, VHM0, VAVH, VZMX, VHZA, VH110, VTPK, VA...","[2004, 2005, 2006, 2007, 2008, 2009, 2010, 201...",2020/06/07 22:00,PdE - Puertos del Estado - Spain
4,28133,Ceuta-coast-buoy,"35,9","-5,33","[DEPH, VHM0, VAVH, VZMX, VHZA, VH110, VTPK, VA...","[1985, 1986, 1987, 1988, 1989, 1990, 1991, 199...",2020/06/07 22:00,PdE - Puertos del Estado - Spain
5,28137,Gijon-coast-buoy,"43,62","-5,66","[DEPH, VHM0, VAVH, VZMX, VHZA, VH110, VTPK, VA...","[2001, 2002, 2003, 2004, 2005, 2006, 2007, 200...",2020/06/07 22:00,PdE - Puertos del Estado - Spain
6,28139,Langosteira-coast-buoy,"43,3494","-8,5635","[DEPH, VHM0, VAVH, VZMX, VHZA, VH110, VTPK, VA...","[2014, 2015, 2016, 2017, 2018, 2019]",2019/10/20 15:00,PdE - Puertos del Estado - Spain
7,28141,LasPalmas-coast-buoy,"28,05","-15,3972","[DEPH, VHM0, VAVH, VZMX, VHZA, VH110, VTPK, VA...","[1992, 1993, 1994, 1995, 1996, 1997, 1998, 199...",2020/06/07 22:00,PdE - Puertos del Estado - Spain
8,28142,Malaga-coast-buoy,"36,66","-4,44","[DEPH, VHM0, VAVH, VZMX, VHZA, VH110, VTPK, VA...","[1985, 1986, 1987, 1988, 1989, 1990, 1991, 199...",2020/06/07 22:00,PdE - Puertos del Estado - Spain
9,28144,Melilla-coast-buoy,"35,319","-2,941","[DEPH, VHM0, VAVH, VZMX, VHZA, VTPK, VAVT, VTZ...","[2008, 2009, 2010, 2011, 2012, 2013, 2014, 201...",2020/06/07 22:00,PdE - Puertos del Estado - Spain


In [13]:
erddap_vtdh= list()
vghs = get_platforms_erddap('VGHS')
df_vtdh = (emodnet_list.loc[emodnet_list.Parameters.apply(lambda x: 'VTDH' in x)])
df_vtdh.reset_index(inplace=True, drop=True)

for i,r in vghs.iterrows():
    if (df_vtdh['PlatformID']==r['PlatformID']).any():
        erddap_vtdh.append(r)
print(len(erddap_vtdh))

#vtdh 

erddap_vtdh= list()
df_vtdh = (emodnet_list.loc[emodnet_list.Parameters.apply(lambda x: 'VTDH' in x)])
df_vtdh.reset_index(inplace=True, drop=True)
vghs = (df_vtdh.loc[df_vtdh.Parameters.apply(lambda x: 'VHZA' in x)])
vghs.reset_index(inplace=True, drop=True)

#print(vghs)

for i,r in df_vtdh.iterrows():
    if (allErddapStations['PlatformID']==r['PlatformID']).any():
        erddap_vtdh.append(r)
print(len(erddap_vtdh))
#for i,r in emodnet_list.loc[('VTDH' in emodnet_list['Parameters']).any].iterrows():


/opt/tljh/user/lib/python3.6/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


0
94


In [18]:
#Comparison of emodnet and erddap stations

height=get_platforms_erddap('VHZA')
#tp=get_platforms_Tp()

#s1 = pd.merge(emodnet_list, height, how='inner')
#print(s1)
        
common_count=0
diff_count=0
dif_list=list()
param_count=0
for i,r in emodnet_list.iterrows():
    if 'VHZA' in r['Parameters']:
        #row=height.loc[str(height['PlatformID']) == r['PlatformID']]
        #if row is None:
        param_count+=1
        if (height['PlatformID']==int(r['PlatformID'])).any():
        #if int(r['PlatformID']) in (height['PlatformID']):
            #print('none')
            #print(r) 
            common_count+=1
            print(r)
        else:
            diff_count+=1
            
    else:
        #diff_count+=1
        dif_list.append(r)
print('For vhza; common points '+str(common_count)+' different points '+str(diff_count)+ ' paramcount '+str(param_count))
print(len(dif_list))

common_ids=0
diff_count=0
dif_list=list()
param_count=0

for i,r in height.iterrows():
    row=None
    #(df['company_name']=='ABC').any()
    if (emodnet_list['PlatformID']==str(r['PlatformID'])).any():
    #if str(r['PlatformID']) in (emodnet_list['PlatformID']):
        common_ids+=1
        row = emodnet_list.loc[emodnet_list['PlatformID'] == (str(r['PlatformID']))]
        #print(str(r['PlatformID'])+row['PlatformID'])
        if 'VHM0' in row['Parameters'].values[0]:
            param_count+=1
        else:
            print(row)
    else:
        diff_count+=1
        dif_list.append(r)
print('For vhm0; common points '+str(common_ids)+' different points '+str(diff_count)+ ' paramcount '+str(param_count))
print(dif_list)

s1 = pd.merge(height, station_list, how='inner', on=['PlatformID'])
print(s1)
common_count=0
diff_count=0
dif_list=list()
for i,r in height.iterrows():
    if r['PlatformID'] in station_list['PlatformID'].values:
        common_count+=1
    else:
        diff_count+=1
        dif_list.append(r)
print('For height; common points '+str(common_count)+' different points '+str(diff_count))
print(dif_list)

/opt/tljh/user/lib/python3.6/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


PlatformID                                                      13795
PlatformName                                                    61499
Latitude                                                      39,4928
Longitude                                                      2,7004
Parameters          [DEPH, ATMP, CNDC, DEWT, DRYT, GSPD, NRAD, PSA...
YearDataMeasured    [2012, 2013, 2014, 2015, 2016, 2017, 2018, 201...
LastDataMeasured                                     2020/06/07 00:30
Institution         SOCIB - Balearic Islands Coastal Observing and...
Name: 678, dtype: object
PlatformID                                                      28126
PlatformName                                     Algeciras-coast-buoy
Latitude                                                      36,0657
Longitude                                                     -5,4149
Parameters          [DEPH, VHM0, VAVH, VZMX, VHZA, VH110, VTPK, VA...
YearDataMeasured                 [2014, 2015, 2016, 2017, 2018, 2

     PlatformID PlatformName Latitude Longitude  \
1227      33153      6201070  43,3501   -8,5628   

                                Parameters YearDataMeasured  LastDataMeasured  \
1227  [DEPH, VDIR, DRYT, WSPD, WDIR, ATMS]           [2017]  2017/11/06 06:00   

      Institution  
1227  Not Defined  
     PlatformID PlatformName    Latitude  Longitude  \
1376     371916     ROLMEDAP  44,2958333  9,1908333   

                                      Parameters YearDataMeasured  \
1376  [DRYT, RELH, WDIR, WSPD, VTDH, VHZA, VGTA]           [2019]   

     LastDataMeasured                                        Institution  
1376             N.D.  ARPAL - UNIGE - Azienda regionale per la prote...  
     PlatformID PlatformName   Latitude  Longitude  \
1501     965440  BatemansBay  -35,70778  150,33694   

                                             Parameters  \
1501  [VZMX, VAVT, VTPK, VGTA, VTZA, VPED, VHZA, VH1...   

                                       YearDataMeasured  LastDataM

NameError: name 'station_list' is not defined